In [34]:
import quandl
quandl.ApiConfig.api_key = "KxKMd8dsHx1sWnaGj6_U"
import numpy as np
import matplotlib as plt
import pandas as pd
import keras
from sklearn.preprocessing import MinMaxScaler

In [35]:
def get_monthly_close_prices(start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SEP',date={'gte':start_date,'lte':end_date}, qopts={'columns':['ticker','date','close']},paginate=True).fillna(value=0)
    close = df.pivot(index='date', columns='ticker', values='close')
    monthly = close.resample('BM', how=lambda x: x[-1])
    #.agg(lambda x: x[-1])
    return monthly

In [ ]:
monthly_prices = get_monthly_close_prices('2018-06-01','2019-01-01')

In [ ]:
def get_monthly_returns(monthly_prices):
    """
    Compute returns for each ticker and date in close.
    
    Parameters
    ----------
    close : DataFrame
        Close prices for each ticker and date
    
    Returns
    -------
    returns : DataFrame
        Returns for each ticker and date
    """
        

    #.agg(lambda x: x[-1])
    monthly_returns = (monthly_prices - monthly_prices.shift(1))/monthly_prices.shift(1).fillna(value=0)
    
    
    return monthly_returns

In [ ]:
monthly_retuns_june_dec_18 = get_monthly_returns(monthly_prices)


In [ ]:
monthly_retuns_june_dec_18.head()

In [ ]:
monthly_retuns_june_dec_18 = monthly_retuns_june_dec_18.iloc[ 1: , : ]

In [ ]:
monthly_retuns_june_dec_18 = monthly_retuns_june_dec_18.fillna(0)

In [ ]:
training_returns = monthly_retuns_june_dec_18.iloc[ :5 , : ]

In [ ]:
target_returns = monthly_retuns_june_dec_18.iloc[ 5:6 , : ]

In [ ]:
def get_factor_data(start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SF1',dimension='ARQ',calendardate={'gte':start_date, 'lte':end_date}, paginate=True).fillna(value=0)

    
    return df

In [ ]:
#quandl.get_table('SHARADAR/SF1',dimension='ARY',calendardate={'gte':'2018-06-01', 'lte':'2019-01-01'}, paginate=True)

In [ ]:
factors_june_dec_18 = get_factor_data('2018-06-01','2019-01-01')

In [ ]:
factors_june_dec_18.head

In [ ]:
monthly_retuns_june_dec_18.head() #['date'].drop_duplicates().sort_values()

In [ ]:
monthly_retuns_june_dec_18.pivot(index='ticker', columns='ticker', values='close')

In [ ]:
factors_june_dec_18['calendardate'].drop_duplicates().sort_values()

In [ ]:
factors_june_dec_18.shape

In [ ]:
import keras.losses
import tensorflow as tf

# Create loss function
def sign_penalty(y_true, y_pred):
    penalty = 100.
    loss = tf.where(tf.less(y_true * y_pred, 0), \
                     penalty * tf.square(y_true - y_pred), \
                     tf.square(y_true - y_pred))

    return tf.reduce_mean(loss, axis=-1)

keras.losses.sign_penalty = sign_penalty  # enable use of loss with keras
print(keras.losses.sign_penalty)

In [ ]:
corr_matrix = df.corr().abs()

In [ ]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper

In [ ]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [ ]:
factors_june_dec_18_reduced = factors_june_dec_18.drop(factors_june_dec_18[to_drop], axis=1)

In [ ]:
factors_june_dec_18_reduced[factors_june_dec_18_reduced['calendardate'] != '2018-12-31'] 

In [ ]:
factors_june_dec_18[factors_june_dec_18['calendardate'] != '2018-12-31']

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
factors_june_dec_18['calendardate'] != '2018-12-31'

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
factors_june_dec_18_reduced['calendardate']

In [ ]:



features = factors_june_dec_18_reduced
target = monthly_retuns_june_dec_18

In [ ]:
features.rename(columns={"calendardate": "date"})

In [ ]:
from sklearn.preprocessing import scale

# Remove unimportant features (weekdays)
train_features = train_features.iloc[:, :-4]
test_features = test_features.iloc[:, :-4]

# Standardize the train and test features
scaled_train_features = scale(train_features)
scaled_test_features = scale(test_features)

# Plot histograms of the 14-day SMA RSI before and after scaling
f, ax = plt.subplots(nrows=2, ncols=1)
train_features.iloc[:, 2].hist(ax=ax[0])
ax[1].hist(scaled_train_features[:, 2])
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Create the model
model_1 = Sequential()
model_1.add(Dense(100, input_dim=scaled_train_features.shape[1], activation='relu'))
model_1.add(Dense(20, activation='relu'))
model_1.add(Dense(1, activation='linear'))

# Fit the model
model_1.compile(optimizer='adam', loss='mse')
history = model_1.fit(scaled_train_features, train_targets, epochs=25)